# Biopython 1: Introduction to Biopython

## Parsing Data

Parsing data is one of the first things you need to do to your data file. Sequence data files contain all kinds of information and metadata, such as sequences themselves, IDs, quality scores, annotations, and more. Parsing a file is the process of reading and extracting that information so that it is in a form where it can be further manipulated or interpreted.

### A parsing example from the official Biopython Tutorial

In this example, we will be parsing files for a Ladyslipper Orchid. This data was obtained by searching the nucleotide database at NCBI and downloading the results in .fasta format. Later on, we will learn how to search for and download data programatically using Biopython.

The data is saved in the folder called `sample-data`. 

In [1]:
from Bio import SeqIO
for seq_record in SeqIO.parse("sample-data/ls_orchid.fasta", "fasta"):
    print(seq_record.id)
    print(seq_record.seq)
    print(len(seq_record))

gi|2765658|emb|Z78533.1|CIZ78533
CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGGAATAAACGATCGAGTGAATCCGGAGGACCGGTGTACTCAGCTCACCGGGGGCATTGCTCCCGTGGTGACCCTGATTTGTTGTTGGGCCGCCTCGGGAGCGTCCATGGCGGGTTTGAACCTCTAGCCCGGCGCAGTTTGGGCGCCAAGCCATATGAAAGCATCACCGGCGAATGGCATTGTCTTCCCCAAAACCCGGAGCGGCGGCGTGCTGTCGCGTGCCCAATGAATTTTGATGACTCTCGCAAACGGGAATCTTGGCTCTTTGCATCGGATGGAAGGACGCAGCGAAATGCGATAAGTGGTGTGAATTGCAAGATCCCGTGAACCATCGAGTCTTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCTTGGGCGTCGCGCTTCGTCTCTCTCCTGCCAATGCTTGCCCGGCATACAGCCAGGCCGGCGTGGTGCGGATGTGAAAGATTGGCCCCTTGTGCCTAGGTGCGGCGGGTCCAAGAGCTGGTGTTTTGATGGCCCGGAACCCGGCAAGAGGTGGACGGATGCTGGCAGCAGCTGCCGTGCGAATCCCCCATGTTGTCGTGCTTGTCGGACAGGCAGGAGAACCCTTCCGAACCCCAATGGAGGGCGGTTGACCGCCATTCGGATGTGACCCCAGGTCAGGCGGGGGCACCCGCTGAGTTTACGC
740
gi|2765657|emb|Z78532.1|CCZ78532
CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGTTGAGACAACAGAATATATGATCGAGTGAATCTGGAGGACCTGTGGTAACTCAGCTCGTCGTGGCACTGCTTTTGTCGTGACCCTGCTTTGTTGTTGGGCCTCCTCAAGAGCTTTCATGGCAGGTTTGAACTTTAGTACGGTGCAGT

Let's "unpack" this code and take a look at some of the objects and functions within Biopython.

`from Bio import SeqIO`

First, we imported `SeqIO` from `Bio`. `Bio` is how you call the Biopython library, and [`SeqIO`](https://biopython.org/wiki/SeqIO) is a module within the Biopython library that deals with importing and exporting sequences, as the name suggests.

Then, we iterate through a for loop to extract the individual sequences contained within the data file. The magic happens in the line:

`SeqIO.parse("sample-data/ls_orchid.fasta", "fasta")`

Here, we use the `parse()` function to parse the data file. The arguments of `parse()` are the file path, as well as the type of file (in this case, fasta). The output of the `parse()` function is a `SeqRecord` object. This object has various properties which can be printed including the sequence `id`, the sequence itself, and the sequence length.

## Managing Memory with Large Data Files

Sequence data files can be extremely large. When we do operations with the data in these files, we want to do so in a way that will not challenge the memory of our computers. The correct way to do so is to access each record individually and perform whichever operations we like on it. In this way we only ever store one record of sequence data at a time in RAM, rather than the entire contents of the data file. 

This can be observed in the previous example. In contrast, an incorrect way to do this would be 

`records = list(SeqIO.parse("file.fa", "fasta")) # Memory Intensive`

or

`record_dict = SeqIO.to_dict(SeqIO.parse("file.fa", "fasta")) # Memory Intensive`

In the above example, each record in the file is turned into an element of a list or dictionary. This means that the entire contents of the data file will be stored in the RAM. 

The drawback of importing each record individually is that it restricts you from performing operations on multiple records at once. To get around this, you can use the `index()` function. Note that this function does not permit all file types.

`record_dict = SeqIO.index("file.fa", "fasta")`

You can then call on the dictionary using the sequence ID as a string as a key to access a given record in the data file. Let's try an example of this using our large data file.

First index the data file `sample-data/ls_orchid.fasta` and print the sequence with the ID "gi|2765654|emb|Z78529.1|CLZ78529". 

In [2]:
record_dict = SeqIO.index("sample-data/ls_orchid.fasta", "fasta")

print(record_dict["gi|2765654|emb|Z78529.1|CLZ78529"].seq)

ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAATCTGGAGGACTTGTGGTTATTTGGCTCGCTAGGGATTTCCTTTTGTGGTGACCATGATTTGTCATTGGGCCTCATTGAGAGCTTTCATGGCGGGTTTGAACCTCTAGCACGGTGCAGTTTGCACCAAGGTATATAAAGAATCACCGATGAATGACATTATTGTCAAAAAAGTCGGAGGTGTGGTGTGTTATTGGTCATGCCAATGAATTGTTGATGACTCTCGCCGAGGGATATCTTGGCTCTTGCATCGATGAAGAATCCCACCGAAATGTGATAAGTGGTGTGAATTGCAGAATCCCGTGAACCATCGAGTCTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCCTGGGCGTCGTATGTTTTATCTCTCCTTCCAATGCTTGTCCAGCATATAGCTAGGCCATCATTGTGTGGATGTGAAAGATTGGCCCCTTGTGCTTAGGTGCGGTGGGTCTAAGGATATGTGTTTTGATGGTCTGAAACTTGGCAAGAGGTGGAGGATGCTGGCAGCCGCAAGGCTATTGTTTGAATCCCCCATGTTGTCATATTTGTTGGGCCTATAGAACAACTTGTTTGGACCCTAATTAAGGCAAAACAATCCTTGGGTGGTTGATTTCCAATCAGATGCGACCCCAGTCAGCGGGCCACCAGCTGAGCTAAAA


## Zipped Data Files

Often when you access large data files they will be compressed. It is possible to keep these files compressed while you parse them. There is a tradeoff to consider if you want to uncompress files before parsing: the uncompressed files can be parsed more quickly, but they will also take up much more memory in your system. Below, we will examine this tradeoff. We have two files: xenoMrna_short.fa (unzipped FASTA file, 500MB) and xenoMrna_short.fa.gz (compressed FASTA file, 130MB). Let's compare the time it takes to parse each.

To generate these files and access the data, run the script in the sample-data folder titled `get_large_data.py`.

First, let's try unzipping then parsing.

In [7]:
import gzip # Import this separately so we don't interfere with our timing experiment

In [11]:
%%timeit -r 2 # Time the execution of this cell

count = 0

with open("sample-data/xenoMrna_short.fa", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        count += 1

print(f"Number of sequences: {count}")       
    

Number of sequences: 500000
Number of sequences: 500000
Number of sequences: 500000
6.45 s ± 204 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


Now, let's parse the file while keeping it zipped.

In [9]:
%%timeit -r 2

count = 0
with gzip.open("sample-data/xenoMrna_short.fa.gz", "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        count += 1

print(f"Number of sequences: {count}")

Number of sequences: 500000
Number of sequences: 500000
Number of sequences: 500000
9.12 s ± 206 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


We can see that it takes longer to parse, but we save on memory. Also, keep in mind that we would still have to take time to unzip the original file even though parsing it will be faster. In each situation you should weigh the memory and time constraints on your system to choose the correct course of action.

## SimpleFastaParser for Large FASTA Files

The Biopython package offers several options for parsing. The default is `parse()`. This provides additional features provided by SeqRecord objects, such as metadata, annotations, and compatibility with various file formats. However, we may want to use a faster option when time is important for large FASTA files. For this, we can use `SimpleFastaParser`. 

`SimpleFastaParser` is a lower-level approach provided by Biopython for parsing FASTA files. It returns tuples containing the sequence identifier and the sequence itself. This method is more memory-efficient and faster for large FASTA files because it doesn't create SeqRecord objects, making it suitable for scenarios where memory usage is a concern.

Let's try this using a larger data file and compare the time each method takes.

In [13]:
%%timeit -r 2

count = 0 

with open("sample-data/xenoMrna_short.fa", "r") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        count += 1

print(f"Number of sequences: {count}")


Number of sequences: 500000
Number of sequences: 500000
Number of sequences: 500000
5.51 s ± 139 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


This should have taken ~5-6 seconds. Now let's try with SimpleFastaParser.

In [14]:
from Bio.SeqIO.FastaIO import SimpleFastaParser

In [16]:
%%timeit -r 2

count = 0 
with open("sample-data/xenoMrna_short.fa") as handle: 
    for seq, seq_id in SimpleFastaParser(handle): # Note that SimpleFastaParser returns a tuple not a SeqRecord object
        count += 1

print(f"Number of sequences: {count}")

Number of sequences: 500000
Number of sequences: 500000
Number of sequences: 500000
3.77 s ± 76 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


This is significantly faster! If you don't need all of the information contained in the Record object, consider using SimpleFastaParser for large FASTA files.

## Sequence Objects

Now that we know how to parse data files, we can think about the basic operations we may want to perform with the sequence data. All of this is done using the [Seq](https://biopython.org/docs/1.75/api/Bio.Seq.html) module. As an example, let's use a record from our ladyslipper file.

`record_dict["gi|2765654|emb|Z78529.1|CLZ78529"]` is a SeqRecord object. It contains the sequence, as well as other information.


In [3]:
record_dict = SeqIO.index("sample-data/ls_orchid.fasta", "fasta") # Indexing our FASTA file

my_record = record_dict["gi|2765654|emb|Z78529.1|CLZ78529"] # Pulling one record from it as a sample

print(type(my_record))
print(my_record)

<class 'Bio.SeqRecord.SeqRecord'>
ID: gi|2765654|emb|Z78529.1|CLZ78529
Name: gi|2765654|emb|Z78529.1|CLZ78529
Description: gi|2765654|emb|Z78529.1|CLZ78529 C.lichiangense 5.8S rRNA gene and ITS1 and ITS2 DNA
Number of features: 0
Seq('ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAA...AAA')


We are only interested in the sequence part. Let's isolate that and print out what type of object it is.

In [4]:
my_sequence = my_record.seq

print(type(my_sequence))
print(my_sequence)

<class 'Bio.Seq.Seq'>
ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAATCTGGAGGACTTGTGGTTATTTGGCTCGCTAGGGATTTCCTTTTGTGGTGACCATGATTTGTCATTGGGCCTCATTGAGAGCTTTCATGGCGGGTTTGAACCTCTAGCACGGTGCAGTTTGCACCAAGGTATATAAAGAATCACCGATGAATGACATTATTGTCAAAAAAGTCGGAGGTGTGGTGTGTTATTGGTCATGCCAATGAATTGTTGATGACTCTCGCCGAGGGATATCTTGGCTCTTGCATCGATGAAGAATCCCACCGAAATGTGATAAGTGGTGTGAATTGCAGAATCCCGTGAACCATCGAGTCTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCCTGGGCGTCGTATGTTTTATCTCTCCTTCCAATGCTTGTCCAGCATATAGCTAGGCCATCATTGTGTGGATGTGAAAGATTGGCCCCTTGTGCTTAGGTGCGGTGGGTCTAAGGATATGTGTTTTGATGGTCTGAAACTTGGCAAGAGGTGGAGGATGCTGGCAGCCGCAAGGCTATTGTTTGAATCCCCCATGTTGTCATATTTGTTGGGCCTATAGAACAACTTGTTTGGACCCTAATTAAGGCAAAACAATCCTTGGGTGGTTGATTTCCAATCAGATGCGACCCCAGTCAGCGGGCCACCAGCTGAGCTAAAA


Now, let's see what kind of operations we can perform on these objects. Note that this is not an exhaustive list, but should cover the main bases.

### String Operations

You can perform basic string operations on Seq objects, like calculating the length, indexing, slicing, and concatenating. 

In [6]:
from Bio.Seq import Seq # Import the Seq module 

print(my_sequence)
print(len(my_sequence)) # Print the length of the sequence

print(my_sequence[2]) # Access individual bases in the sequence with 0-indexing
print(my_sequence[2:10]) # Access slices of the sequence

other_sequence = Seq("ACTG") # Create a new Seq object from a string

new_sequence = other_sequence + my_sequence
print(new_sequence) # Concatenate Seq objects

ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAATCTGGAGGACTTGTGGTTATTTGGCTCGCTAGGGATTTCCTTTTGTGGTGACCATGATTTGTCATTGGGCCTCATTGAGAGCTTTCATGGCGGGTTTGAACCTCTAGCACGGTGCAGTTTGCACCAAGGTATATAAAGAATCACCGATGAATGACATTATTGTCAAAAAAGTCGGAGGTGTGGTGTGTTATTGGTCATGCCAATGAATTGTTGATGACTCTCGCCGAGGGATATCTTGGCTCTTGCATCGATGAAGAATCCCACCGAAATGTGATAAGTGGTGTGAATTGCAGAATCCCGTGAACCATCGAGTCTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCCTGGGCGTCGTATGTTTTATCTCTCCTTCCAATGCTTGTCCAGCATATAGCTAGGCCATCATTGTGTGGATGTGAAAGATTGGCCCCTTGTGCTTAGGTGCGGTGGGTCTAAGGATATGTGTTTTGATGGTCTGAAACTTGGCAAGAGGTGGAGGATGCTGGCAGCCGCAAGGCTATTGTTTGAATCCCCCATGTTGTCATATTTGTTGGGCCTATAGAACAACTTGTTTGGACCCTAATTAAGGCAAAACAATCCTTGGGTGGTTGATTTCCAATCAGATGCGACCCCAGTCAGCGGGCCACCAGCTGAGCTAAAA
733
G
GGCGAGCT
ACTGACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAATCTGGAGGACTTGTGGTTATTTGGCTCGCTAGGGATTTCCTTTTGTGGTGACCATGATTTGTCATTGGGCCTCATTGAGAGCTTTCATGGCGGGTTTGAACCTCTAGCACGGTGCAGTTTGCACCAAGGTATATAAAGAATCACCGATGAATGACATTATTGTCAAAAAAGTCGGAGGTGTGGTGTGTTATTG

### Central Dogma and Sequence Operations

 You can also execute Central Dogma operations such as transcription on sequences, as well as other operations such as reverse complements. While these operations can be manually coded, it is less error-prone and easier to use the built-in functions provided by Biopython.

In [29]:
short_sequence = my_sequence[:9] # Shorten for clarity
print(short_sequence)

sequence_rna = short_sequence.transcribe() # Transcribe to RNA
print(sequence_rna)

sequence_protein = short_sequence.translate() # Direct transcription to protein
print(sequence_protein) 

sequence_revcomp = short_sequence.reverse_complement() # Reverse complement
print(sequence_revcomp)


ACGGCGAGC
ACGGCGAGC
TAS
GCTCGCCGT


### Searching within Sequences

You can check whether a given sub-sequence exists within your sequence in the following manner:

In [19]:
is_present_1 = "ACTG" in my_sequence # Returns True or False 
is_present_2 = "ACGG" in my_sequence

print(is_present_1)
print(is_present_2)

False
True


### Counting

You might want to count the occurences of a certain base, or of a sequence of bases. There are two ways to do this, depending on whether you want your count to be overlapping. 

In [35]:
print(my_sequence)

count_t = my_sequence.count("T") # Count the number of times T occurs
print(count_t)

count_tt = my_sequence.count("TT") # Non-overlapping count of the times TT occurs
print(count_tt)

count_tt_over = my_sequence.count_overlap("TT") # Overlappig count of the times TT occurs
print(count_tt_over)


ACGGCGAGCTGCCGAAGGACATTGTTGAGACAGCAGAATATACGATTGAGTGAATCTGGAGGACTTGTGGTTATTTGGCTCGCTAGGGATTTCCTTTTGTGGTGACCATGATTTGTCATTGGGCCTCATTGAGAGCTTTCATGGCGGGTTTGAACCTCTAGCACGGTGCAGTTTGCACCAAGGTATATAAAGAATCACCGATGAATGACATTATTGTCAAAAAAGTCGGAGGTGTGGTGTGTTATTGGTCATGCCAATGAATTGTTGATGACTCTCGCCGAGGGATATCTTGGCTCTTGCATCGATGAAGAATCCCACCGAAATGTGATAAGTGGTGTGAATTGCAGAATCCCGTGAACCATCGAGTCTTTGAACGCAAGTTGCGCCCGAGGCCATCAGGCTAAGGGCACGCCTGCCTGGGCGTCGTATGTTTTATCTCTCCTTCCAATGCTTGTCCAGCATATAGCTAGGCCATCATTGTGTGGATGTGAAAGATTGGCCCCTTGTGCTTAGGTGCGGTGGGTCTAAGGATATGTGTTTTGATGGTCTGAAACTTGGCAAGAGGTGGAGGATGCTGGCAGCCGCAAGGCTATTGTTTGAATCCCCCATGTTGTCATATTTGTTGGGCCTATAGAACAACTTGTTTGGACCCTAATTAAGGCAAAACAATCCTTGGGTGGTTGATTTCCAATCAGATGCGACCCCAGTCAGCGGGCCACCAGCTGAGCTAAAA
208
48
62


### Example: GC Content 

Here is an example of how to use the count utility to find the sequence in our data file with the highest GC content. The GC content is defined as 

$\frac{G + C}{length} \times 100 \%$

In [39]:
max_seq_id = None # Define variables for the sequence ID of the sequence with the highest GC content
max_gc_content = 0 # Define highest GC content, 0 as a placeholder

record_dict = SeqIO.index("sample-data/ls_orchid.fasta", "fasta") # Index the file for later

for seq_record in SeqIO.parse("sample-data/ls_orchid.fasta", "fasta"): # Do things the right way
    sequence = seq_record.seq # Convert to a string
    seq_id = seq_record.id
    gc_content = (sequence.count("C") + sequence.count("G")) * 100 / len(sequence) # CAlculate GC content as a percentage

    if gc_content > max_gc_content: # Update the variables as needed
        max_seq_id = seq_id
        max_gc_content = gc_content

print(max_seq_id)
print(max_gc_content)
print("Full Sequence Details:")
print(record_dict[max_seq_id])

gi|2765658|emb|Z78533.1|CIZ78533
59.5945945945946
Full Sequence Details:
ID: gi|2765658|emb|Z78533.1|CIZ78533
Name: gi|2765658|emb|Z78533.1|CIZ78533
Description: gi|2765658|emb|Z78533.1|CIZ78533 C.irapeanum 5.8S rRNA gene and ITS1 and ITS2 DNA
Number of features: 0
Seq('CGTAACAAGGTTTCCGTAGGTGAACCTGCGGAAGGATCATTGATGAGACCGTGG...CGC')


## Writing Seq Files

We have already covered the basics of the "I" in SeqIO, now let's cover the "O" (out). The first thing we might want to do is write a sequence to a data file using the `write()` function.

In [48]:
from Bio.SeqRecord import SeqRecord

# Define a function that takes in the required parameters and returns a SeqRecord object

def make_record(seq, id, name, description): 
    return SeqRecord(seq=seq, id = id, name = name, description=description)

# Define the information we want to write in our FASTA file
# This is a bit unrealistic but you could imagine several different ways of generating this information

seq_1 = Seq("ACGTACGATCAGTAGGCATCAGTCACTACT")
seq_2 = Seq("ACTCAATGGGATCATGTACTGACTACGTAC")

id_1 = "Seq1"
id_2 = "Seq2"

name_1 = "Sequence 1"
name_2 = "Sequence 2"

desc_1 = "This is a description of Sequence 1"
desc_2 = "This is a description of Sequence 2"

# Generate records from the information
rec_1 = make_record(seq=seq_1, id=id_1, name=name_1, description=desc_1)
rec_2 = make_record(seq=seq_2, id=id_2, name=name_2, description=desc_2)

recs = [rec_1, rec_2] # Put our records in a list

# Write records in file
with open("writing_sample.fasta", "w") as output_handle:
    for record in recs:
        SeqIO.write(record, output_handle, "fasta")

To check our work, let's parse this file.

In [51]:
for seq_record in SeqIO.parse("writing_sample.fasta", "fasta"):
    print(seq_record)

ID: Seq1
Name: Seq1
Description: Seq1 This is a description of Sequence 1
Number of features: 0
Seq('ACGTACGATCAGTAGGCATCAGTCACTACT')
ID: Seq2
Name: Seq2
Description: Seq2 This is a description of Sequence 2
Number of features: 0
Seq('ACTCAATGGGATCATGTACTGACTACGTAC')


## Changing File Formats

One use of the SeqIO package is to read data from one file format and then convert it to a different file format. You can do this the long way, by parsing and then writing SeqRecord objects to a different file format, or you can use the built-in `convert()` function, althought this is not supported by all file types. Let's say we want to take the FASTA file we just wrote and convert it to GenBank format. 

In [17]:
with open("writing_sample.fasta", "r") as input_handle:
    with open("writing_sample.gb", "w") as output_handle:
        for seq_record in SeqIO.parse(input_handle, "fasta"): # Iterate through records in the input file
            seq_record.annotations['molecule_type'] = 'DNA' # You can optionally add other info for the gb format here. For .gb files molecule type is required.
            SeqIO.write(seq_record, output_handle, "genbank") # Write record in gb format to the new data file


Again, let's parse to check the information.

In [18]:
for seq_record in SeqIO.parse("writing_sample.gb", "genbank"): # Parse the new file in the normal way, specifying new format
    print(seq_record)

ID: Seq1
Name: Seq1
Description: Seq1 This is a description of Sequence 1
Number of features: 0
/molecule_type=DNA
/data_file_division=UNK
/date=01-JAN-1980
/accessions=['Seq1']
/keywords=['']
/source=
/organism=.
/taxonomy=[]
Seq('ACGTACGATCAGTAGGCATCAGTCACTACT')
ID: Seq2
Name: Seq2
Description: Seq2 This is a description of Sequence 2
Number of features: 0
/molecule_type=DNA
/data_file_division=UNK
/date=01-JAN-1980
/accessions=['Seq2']
/keywords=['']
/source=
/organism=.
/taxonomy=[]
Seq('ACTCAATGGGATCATGTACTGACTACGTAC')


## Useful Links and Documentation

* [Biopython Documentation](https://biopython.org/wiki/Documentation)
* [SeqIO](https://biopython.org/wiki/SeqIO)
* [SeqRecord Class](https://biopython.org/wiki/SeqRecord)

## Credits and Inspiration

* [Biopython Tutorial and Cookbook](https://biopython.org/DIST/docs/tutorial/Tutorial.html)
* [Lana Dominkovic's Biopython Tutorials](https://github.com/lanadominkovic/12-days-of-biopython) (also check out her [YouTube channel](https://www.youtube.com/@LanaDominkovic))

